In [4]:
import json

import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

### Load data

In [5]:
pdbbind_training_set_pk = pd.read_csv('../data/pdbbind_training_set_binding_affinity.csv', index_col=0, header=None, squeeze=True)

crystal_pose_features = pd.read_csv('../data/crystal_pose_features.csv', index_col=0)
minimised_pose_features = pd.read_csv('../data/minimised_pose_features.csv', index_col=0)
docked_pose_features = pd.read_csv('../data/docked_pose_features.csv', index_col=0)

feature_sets = {}
with open('../data/lb_feature_names.txt') as f:
    feature_sets['LB'] = pd.Index([l.strip() for l in f])
with open('../data/sb_feature_names.txt') as f:
    feature_sets['SB'] = pd.Index([l.strip() for l in f])
with open('../data/hb_feature_names.txt') as f:
    feature_sets['HB'] = pd.Index([l.strip() for l in f])

# We've enumerated the docked poses associated to each PDB structure - these labels are used for cross-validation later
with open('../data/docked_pose_labels.json') as f:
    docked_pose_labels = json.load(f)


### Prepare cross-validation folds

Randomly shuffle and split into five folds - we'll use the same folds across all experiments.

In [6]:
shuffled = pdbbind_training_set_pk.sample(frac=1, replace=False, random_state=42).index
n_test = int(len(shuffled) / 5)
folds = [shuffled[:n_test], shuffled[n_test:2*n_test], shuffled[2*n_test:3*n_test], shuffled[3*n_test:4*n_test], shuffled[4*n_test:]]

pdbbind_training_set = pdbbind_training_set_pk.index

### Performance of Vina scoring function on CV folds

In [47]:
vina_crystal_pk = pd.read_csv('../results/vina_crystal_predicted_pk.csv', index_col=0, squeeze=True)
vina_docked_pk = pd.read_csv('../results/vina_docked_predicted_pk.csv', index_col=0, squeeze=True)
vina_docked_mean_pk = pd.read_csv('../results/vina_docked_mean_predicted_pk.csv', index_col=0, squeeze=True)
vina_minimized_pk = pd.read_csv('../results/vina_minimized_predicted_pk.csv', index_col=0, squeeze=True)

vina_crystal_fold_pearsonr = []
vina_docked_fold_pearsonr = []
vina_mean_docked_fold_pearsonr = []
vina_minimized_fold_pearsonr = []

for fold in folds:
    y_true = pdbbind_training_set_pk.loc[fold]
    # crystal
    y_pred = vina_crystal_pk.loc[fold]
    vina_crystal_fold_pearsonr.append(stats.pearsonr(y_true, y_pred)[0])
    # docked
    y_pred = vina_docked_pk.loc[fold]
    vina_docked_fold_pearsonr.append(stats.pearsonr(y_true, y_pred)[0])
    # docked (mean)
    y_pred = vina_docked_mean_pk.loc[fold]
    vina_mean_docked_fold_pearsonr.append(stats.pearsonr(y_true, y_pred)[0])
    # minimized
    y_pred = vina_minimized_pk.loc[fold]
    vina_minimized_fold_pearsonr.append(stats.pearsonr(y_true, y_pred)[0])
vina_crystal_cv_pearsonr = np.mean(vina_crystal_fold_pearsonr)
vina_docked_cv_pearsonr = np.mean(vina_docked_fold_pearsonr)
vina_mean_docked_cv_pearsonr = np.mean(vina_mean_docked_fold_pearsonr)
vina_minimized_cv_pearsonr = np.mean(vina_minimized_fold_pearsonr)
vina_crystal_cv_pearsonr_stdev = np.std(vina_crystal_fold_pearsonr)
vina_docked_cv_pearsonr_stdev = np.std(vina_docked_fold_pearsonr)
vina_mean_docked_cv_pearsonr_stdev = np.std(vina_mean_docked_fold_pearsonr)
vina_minimized_cv_pearsonr_stdev = np.std(vina_minimized_fold_pearsonr)

In [48]:
print(vina_crystal_cv_pearsonr, vina_crystal_cv_pearsonr_stdev)
print(vina_docked_cv_pearsonr, vina_docked_cv_pearsonr_stdev)
print(vina_mean_docked_cv_pearsonr, vina_mean_docked_cv_pearsonr_stdev)
print(vina_minimized_cv_pearsonr, vina_minimized_cv_pearsonr_stdev)

0.38073200219877706 0.05510186711706739
0.4510107462911782 0.021211050646424593
0.4429890358698632 0.026997838408360817
0.4581152455382108 0.03816169605968634


# Cross-validation using crystal poses

For reference, we first establish a benchmark by performing the cross-validation experiment using crystal poses for training and testing.

In [16]:
cv_crystal_results = {}

for model in feature_sets:
    fold_pearsonr = []
    fold_mse = []
    for fold in folds:
        index_train = pdbbind_training_set.difference(fold)
        X_train = crystal_pose_features.loc[index_train, feature_sets[model]].values
        X_test = crystal_pose_features.loc[fold, feature_sets[model]].values
        y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()
        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)
        pred = rf.predict(X_test)
        fold_pearsonr.append(stats.pearsonr(y_test, pred)[0])
        fold_mse.append(mean_squared_error(y_test, pred))
    cv_crystal_results[model] = {'pearsonr': np.mean(fold_pearsonr), 'rmse': np.sqrt(np.mean(fold_mse)), 'pearsonr_stdev': np.std(fold_pearsonr)}

with open('../results/cv_crystal_results.json', 'w') as f:
    json.dump(cv_crystal_results, f)

In [19]:
cv_crystal_results

{'LB': {'pearsonr': 0.7189989629009104,
  'rmse': 1.3573476912595055,
  'pearsonr_stdev': 0.010191059223340043},
 'SB': {'pearsonr': 0.7469087741840121,
  'rmse': 1.3017764256081878,
  'pearsonr_stdev': 0.01346888659041876},
 'HB': {'pearsonr': 0.7675207374865701,
  'rmse': 1.25673188463707,
  'pearsonr_stdev': 0.011563595080428276}}

In [17]:
cv_minimised_results = {}

for model in feature_sets:
    fold_pearsonr = []
    fold_mse = []
    for fold in folds:
        index_train = pdbbind_training_set.difference(fold)
        X_train = minimised_pose_features.loc[index_train, feature_sets[model]].values
        X_test = minimised_pose_features.loc[fold, feature_sets[model]].values
        y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()
        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)
        pred = rf.predict(X_test)
        fold_pearsonr.append(stats.pearsonr(y_test, pred)[0])
        fold_mse.append(mean_squared_error(y_test, pred))
    cv_minimised_results[model] = {'pearsonr': np.mean(fold_pearsonr), 'rmse': np.sqrt(np.mean(fold_mse)), 'pearsonr_stdev': np.std(fold_pearsonr)}

with open('../results/cv_minimised_results.json', 'w') as f:
    json.dump(cv_minimised_results, f)

In [18]:
cv_minimised_results

{'LB': {'pearsonr': 0.7189989629009104,
  'rmse': 1.3573476912595055,
  'pearsonr_stdev': 0.010191059223340043},
 'SB': {'pearsonr': 0.7443872424537414,
  'rmse': 1.3071651954367485,
  'pearsonr_stdev': 0.01499707806772012},
 'HB': {'pearsonr': 0.765835153939275,
  'rmse': 1.2607803087652687,
  'pearsonr_stdev': 0.010610296025747914}}

### Cross-validation using minimised poses

We repeat the benchmarking exercise, this time using minimized poses of each ligand for training and testing.

### PLEC

In [84]:
from oddt.fingerprints import sparse_to_csr_matrix, csr_matrix_to_sparse, fold, sparse_to_dense
from scipy import sparse

with open('../data/pdbbind_training_set.txt') as f:
    train_pdbs = [l.strip() for l in f]

with open('pdbbind_training_set_docked_plec_sparse.json') as f:
    train_features_docked = json.load(f)
    
plec_docked = {pdb: feats for pdb, feats in zip(train_pdbs, train_features_docked)}
    
with open('pdbbind_training_set_crystal_plec_sparse.json') as f:
    train_features_crystal = json.load(f)
    
plec_crystal = {pdb: feats for pdb, feats in zip(train_pdbs, train_features_crystal)}

In [96]:
%%time
plec_crystal_fold_predicted = []
plec_docked_fold_predicted = []
plec_fold_crystal_pearsonr = []
plec_fold_docked_pearsonr = []

for i, fold in enumerate(folds):
    print(f'Fold {i+1}')
    fold_train = pdbbind_training_set.difference(fold)
    y_train = pdbbind_training_set_pk.loc[fold_train].values.ravel()
    y_test = pdbbind_training_set_pk.loc[fold].values.ravel()
    training_features = [plec_crystal[pdb] for pdb in fold_train]
    test_features = [plec_crystal[pdb] for pdb in fold]
    X_train = sparse.vstack([sparse_to_csr_matrix(plec, size=65536) for plec in training_features], format='csr')
    X_test = sparse.vstack([sparse_to_csr_matrix(plec, size=65536) for plec in test_features], format='csr')
    rf = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=6)
    print('Crystal')
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    rp = stats.pearsonr(y_test, pred)[0]
    plec_fold_crystal_pearsonr.append(rp)
    plec_crystal_fold_predicted.append({pdb: score for pdb, score in zip(fold, pred)})
    
    training_features = [plec_docked[pdb] for pdb in fold_train]
    test_features = [plec_docked[pdb] for pdb in fold]
    X_train = sparse.vstack([sparse_to_csr_matrix(plec, size=65536) for plec in training_features], format='csr')
    X_test = sparse.vstack([sparse_to_csr_matrix(plec, size=65536) for plec in test_features], format='csr')
    rf = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=6)
    print('Docked')
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    rp = stats.pearsonr(y_test, pred)[0]
    plec_fold_docked_pearsonr.append(rp)
    plec_docked_fold_predicted.append({pdb: score for pdb, score in zip(fold, pred)})

Fold 1
Crystal
Docked
Fold 2
Crystal
Docked
Fold 3
Crystal
Docked
Fold 4
Crystal
Docked
Fold 5
Crystal
Docked
CPU times: user 11h 30min 20s, sys: 13.8 s, total: 11h 30min 34s
Wall time: 1h 56min 8s


In [97]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open('cross_validation_plec_crystal_predictions.json', 'w') as f:
    json.dump(plec_crystal_fold_predicted, f, cls=NumpyEncoder)
    
with open('cross_validation_plec_docked_predictions.json', 'w') as f:
    json.dump(plec_docked_fold_predicted, f, cls=NumpyEncoder)

In [101]:
np.mean(plec_fold_crystal_pearsonr), np.std(plec_fold_crystal_pearsonr)

(0.7520953642863558, 0.023106832017653945)

In [100]:
np.mean(plec_fold_docked_pearsonr), np.std(plec_fold_docked_pearsonr)

(0.6902568965221363, 0.019819736437548422)

In [105]:
lb_fold_docked_predicted = []

for fold in folds:
    index_train = pdbbind_training_set.difference(fold)

    X_train = lb_features.loc[index_train].values
    y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()

    rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
    rf.fit(X_train, y_train)

    X_test = lb_features.loc[fold].values
    y_test = pdbbind_training_set_pk.loc[fold].values.ravel()

    pred = pd.Series(data=rf.predict(X_test), index=fold)
    lb_fold_docked_predicted.append(pred)

In [114]:
fold_rps = []
for plec_fold, lb_fold in zip(plec_docked_fold_predicted, lb_fold_docked_predicted):
    pred = pd.Series(plec_fold).loc[lb_fold.index]
    stacked_pred = pd.Series({pdb: (pred[pdb] + lb_fold[pdb])/2 for pdb in pred.index})
    y_true = pdbbind_training_set_pk.loc[pred.index]
    fold_rps.append(stats.pearsonr(y_true, stacked_pred)[0])
np.mean(fold_rps), np.std(fold_rps)

In [115]:
np.mean(fold_rps), np.std(fold_rps)

(0.740611193740369, 0.015771925742579095)

In [116]:
fold_rps = []
for plec_fold, lb_fold in zip(plec_crystal_fold_predicted, lb_fold_docked_predicted):
    pred = pd.Series(plec_fold).loc[lb_fold.index]
    stacked_pred = pd.Series({pdb: (pred[pdb] + lb_fold[pdb])/2 for pdb in pred.index})
    y_true = pdbbind_training_set_pk.loc[pred.index]
    fold_rps.append(stats.pearsonr(y_true, stacked_pred)[0])
np.mean(fold_rps), np.std(fold_rps)

(0.7627133000723262, 0.016413776748954015)

In [ ]:
cv_docked_results = {}

for model in ['SB' ,'LB']
    fold_results = []
    for fold in folds:
        index_train = pdbbind_training_set.difference(fold)
        training_pose_labels = []

        # Get the labels for the highest-ranked pose for each training complex
        for pdb in index_train:
            training_pose_labels.append(docked_pose_labels[pdb][0])
        training_pose_labels = pd.Index(training_pose_labels)

        X_train = docked_pose_features.loc[training_pose_labels, feature_sets[model]].values
        y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()

        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)

        # List docked poses for this fold
        fold_dock_labels = []
        for pdb in fold:
            fold_dock_labels.extend(docked_pose_labels[pdb])
        fold_dock_labels = pd.Index(fold_dock_labels)

        X_test = docked_pose_features.loc[fold_dock_labels, feature_sets[model]].values
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()

        pred = pd.Series(data=rf.predict(X_test), index=fold_dock_labels)

### Experiment 1 - scoring strategy

First we run the cross-validation experiment using different strategies for scoring a ligand when multiple docked poses are available. Three strategies were tested: scoring the pose ranked highest by Smina ("top dock"); scoring all poses and taxing the highest score ("all docks max"); and scoring all poses and taking the mean score ("all docks mean"). Models are trained using a single pose for each ligand, minimised using Smina to achieve a single near-native docked pose. We also train and test using crystal poses for reference.

In [8]:
cv_train_minimized_test_top_dock_results = {}
cv_train_minimized_test_all_docks_max_results = {}
cv_train_minimized_test_all_docks_mean_results = {}

for model in feature_sets:
    fold_pearsonr_max = []
    fold_pearsonr_mean = []
    fold_pearsonr_top = []
    fold_mse_max = []
    fold_mse_mean = []
    fold_mse_top = []
    for fold in folds:
        index_train = pdbbind_training_set.difference(fold)
        X_train = minimised_pose_features.loc[index_train, feature_sets[model]].values
        y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()
        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)

        # List docked poses for this fold
        fold_dock_labels = []
        for pdb in fold:
            fold_dock_labels.extend(docked_pose_labels[pdb])
        fold_dock_labels = pd.Index(fold_dock_labels)

        X_test = docked_pose_features.loc[fold_dock_labels, feature_sets[model]].values
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()

        pred = pd.Series(data=rf.predict(X_test), index=fold_dock_labels)

        # Score all poses, taking max/mean score for each ligand
        max_pred = []
        mean_pred = []

        for pdb in fold:
            max_pred.append(np.max(pred.loc[docked_pose_labels[pdb]]))
            mean_pred.append(np.mean(pred.loc[docked_pose_labels[pdb]]))
        fold_pearsonr_max.append(stats.pearsonr(y_test, max_pred)[0])
        fold_pearsonr_mean.append(stats.pearsonr(y_test, mean_pred)[0])
        fold_mse_max.append(mean_squared_error(y_test, max_pred))
        fold_mse_mean.append(mean_squared_error(y_test, mean_pred))

        # Take the score of the pose ranked highest by Smina
        top_pred = []
        for pdb in fold:
            top_pred.append(pred.loc[docked_pose_labels[pdb][0]])
        fold_pearsonr_top.append(stats.pearsonr(y_test, top_pred)[0])
        fold_mse_top.append(mean_squared_error(y_test, top_pred))
    
    cv_train_minimized_test_all_docks_max_results[model] = {'pearsonr': np.mean(fold_pearsonr_max), 
                                                            'pearsonr_stdev': np.std(fold_pearsonr_max), 
                                                            'rmse': np.sqrt(np.mean(fold_mse_max))}
    
    cv_train_minimized_test_all_docks_mean_results[model] = {'pearsonr': np.mean(fold_pearsonr_mean), 
                                                             'pearsonr_stdev': np.std(fold_pearsonr_mean),
                                                             'rmse': np.sqrt(np.mean(fold_mse_mean))}
    cv_train_minimized_test_top_dock_results[model] = {'pearsonr': np.mean(fold_pearsonr_top), 
                                                       'pearsonr_stdev': np.std(fold_pearsonr_top),
                                                       'rmse': np.sqrt(np.mean(fold_mse_top))}

In [22]:
test_strategy_pearsonr = {
    'Smina top pose': {model: cv_train_minimized_test_top_dock_results[model]['pearsonr'] for model in feature_sets},
    'Maximum pose score': {model: cv_train_minimized_test_all_docks_max_results[model]['pearsonr'] for model in feature_sets},
    'Mean pose score': {model: cv_train_minimized_test_all_docks_mean_results[model]['pearsonr'] for model in feature_sets},
    'Train-test crystal': {m: cv_crystal_results[m]['pearsonr'] for m in feature_sets}
}
test_strategy_pearsonr = pd.DataFrame(test_strategy_pearsonr).loc[['LB','SB','HB']]
test_strategy_pearsonr.index = ['LB model', 'SB model','HB model']
test_strategy_pearsonr.T.to_csv('../results/train_minimised_pose_cv_pearsonr.csv')
test_strategy_pearsonr.T

,LB model,SB model,HB model
Smina top pose,0.718999,0.659097,0.713919
Maximum pose score,0.718999,0.682261,0.732412
Mean pose score,0.718999,0.604533,0.676920
Train-test crystal,0.718999,0.746909,0.767521


In [27]:
test_strategy_pearsonr_stdev = {
    'Smina top pose': {model: cv_train_minimized_test_top_dock_results[model]['pearsonr_stdev'] for model in feature_sets},
    'Maximum pose score': {model: cv_train_minimized_test_all_docks_max_results[model]['pearsonr_stdev'] for model in feature_sets},
    'Mean pose score': {model: cv_train_minimized_test_all_docks_mean_results[model]['pearsonr_stdev'] for model in feature_sets},
    'Train-test crystal': {m: cv_crystal_results[m]['pearsonr_stdev'] for m in feature_sets}
}
test_strategy_pearsonr_stdev = pd.DataFrame(test_strategy_pearsonr_stdev).loc[['LB','SB','HB']]
test_strategy_pearsonr_stdev.index = ['LB model', 'SB model','HB model']
test_strategy_pearsonr_stdev.T

,LB model,SB model,HB model
Smina top pose,0.010191,0.017338,0.015575
Maximum pose score,0.010191,0.014342,0.009854
Mean pose score,0.010191,0.011656,0.011342
Train-test crystal,0.010191,0.013469,0.011564


### Experiment 2 - Training on docked poses

Next, we repeat the cross-validation experiment, this itme training on the docked pose ranked highest by Smina for each ligand. 

In [10]:
cv_train_top_dock_test_top_dock_results = {}
cv_train_top_dock_test_all_docks_max_results = {}
cv_train_top_dock_test_all_docks_mean_results = {}

for model in feature_sets:
    fold_pearsonr_max = []
    fold_pearsonr_mean = []
    fold_pearsonr_top = []
    fold_mse_max = []
    fold_mse_mean = []
    fold_mse_top = []
    for fold in folds:
        index_train = pdbbind_training_set.difference(fold)
        training_pose_labels = []

        # Get the labels for the highest-ranked pose for each training complex
        for pdb in index_train:
            training_pose_labels.append(docked_pose_labels[pdb][0])
        training_pose_labels = pd.Index(training_pose_labels)

        X_train = docked_pose_features.loc[training_pose_labels, feature_sets[model]].values
        y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()

        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)

        # List docked poses for this fold
        fold_dock_labels = []
        for pdb in fold:
            fold_dock_labels.extend(docked_pose_labels[pdb])
        fold_dock_labels = pd.Index(fold_dock_labels)

        X_test = docked_pose_features.loc[fold_dock_labels, feature_sets[model]].values
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()

        pred = pd.Series(data=rf.predict(X_test), index=fold_dock_labels)

        # Score all poses, taking max/mean score for each ligand
        max_pred = []
        mean_pred = []

        for pdb in fold:
            max_pred.append(np.max(pred.loc[docked_pose_labels[pdb]]))
            mean_pred.append(np.mean(pred.loc[docked_pose_labels[pdb]]))
        fold_pearsonr_max.append(stats.pearsonr(y_test, max_pred)[0])
        fold_pearsonr_mean.append(stats.pearsonr(y_test, mean_pred)[0])
        fold_mse_max.append(mean_squared_error(y_test, max_pred))
        fold_mse_mean.append(mean_squared_error(y_test, mean_pred))

        # Take the score of the pose ranked highest by Smina
        top_pred = []
        for pdb in fold:
            top_pred.append(pred.loc[docked_pose_labels[pdb][0]])
        fold_pearsonr_top.append(stats.pearsonr(y_test, top_pred)[0])
        fold_mse_top.append(mean_squared_error(y_test, top_pred))

    cv_train_top_dock_test_all_docks_max_results[model] = {'pearsonr': np.mean(fold_pearsonr_max), 
                                                            'pearsonr_stdev': np.std(fold_pearsonr_max), 
                                                            'rmse': np.sqrt(np.mean(fold_mse_max))}
    
    cv_train_top_dock_test_all_docks_mean_results[model] = {'pearsonr': np.mean(fold_pearsonr_mean), 
                                                            'pearsonr_stdev': np.std(fold_pearsonr_mean), 
                                                            'rmse': np.sqrt(np.mean(fold_mse_mean))}
    cv_train_top_dock_test_top_dock_results[model] = {'pearsonr': np.mean(fold_pearsonr_top), 
                                                      'pearsonr_stdev': np.std(fold_pearsonr_top), 
                                                      'rmse': np.sqrt(np.mean(fold_mse_top))}

In [11]:
test_strategy_pearsonr = {
    'Smina top pose': {model: cv_train_top_dock_test_top_dock_results[model]['pearsonr'] for model in feature_sets},
    'Maximum pose score': {model: cv_train_top_dock_test_all_docks_max_results[model]['pearsonr'] for model in feature_sets},
    'Mean pose score': {model: cv_train_top_dock_test_all_docks_mean_results[model]['pearsonr'] for model in feature_sets},
    'Train-test crystal': {m: cv_crystal_results[m]['pearsonr'] for m in feature_sets}
}
test_strategy_pearsonr = pd.DataFrame(test_strategy_pearsonr).loc[['LB','SB','HB']]
test_strategy_pearsonr.index = ['LB model', 'SB model','HB model']
test_strategy_pearsonr.T.to_csv('../results/train_top_docked_pose_cv_pearsonr.csv')
test_strategy_pearsonr.T

,LB model,SB model,HB model
Smina top pose,0.718999,0.676220,0.738276
Maximum pose score,0.718999,0.687132,0.744309
Mean pose score,0.718999,0.643699,0.725241
Train-test crystal,0.718999,0.746909,0.767521


In [28]:
test_strategy_pearsonr_stdev = {
    'Smina top pose': {model: cv_train_top_dock_test_top_dock_results[model]['pearsonr_stdev'] for model in feature_sets},
    'Maximum pose score': {model: cv_train_top_dock_test_all_docks_max_results[model]['pearsonr_stdev'] for model in feature_sets},
    'Mean pose score': {model: cv_train_top_dock_test_all_docks_mean_results[model]['pearsonr_stdev'] for model in feature_sets},
    'Train-test crystal': {m: cv_crystal_results[m]['pearsonr_stdev'] for m in feature_sets}
}
test_strategy_pearsonr_stdev = pd.DataFrame(test_strategy_pearsonr_stdev).loc[['LB','SB','HB']]
test_strategy_pearsonr_stdev.index = ['LB model', 'SB model','HB model']
test_strategy_pearsonr_stdev.T

,LB model,SB model,HB model
Smina top pose,0.010191,0.016913,0.013689
Maximum pose score,0.010191,0.013228,0.009312
Mean pose score,0.010191,0.010142,0.012920
Train-test crystal,0.010191,0.013469,0.011564


### Experiment 3 - Training using multiple poses

Next we again repeat the cross-validation experiment, this time training on all of the docked poses for each ligand. To control for the effect of increasing the size of training set, we also repeat the experiment by training on a number of redundant copies of the top pose for each ligand equal to the number of docked poses.

In [35]:
# run cv on docks, training on all docks
cv_train_all_docks_test_all_docks_results = {}

for model in feature_sets:
    fold_pearsonr = []
    fold_mse = []
    for fold in folds:
        index_train = pdbbind_training_set.difference(fold)
        
        training_pose_labels = []
        for pdb in index_train:
            training_pose_labels.extend(docked_pose_labels[pdb])
        training_pose_labels = pd.Index(training_pose_labels)

        X_train = docked_pose_features.loc[training_pose_labels, feature_sets[model]].values
        # Training affinities are the same for each pose of a ligand
        training_pose_pdbs = pd.Index(i[:4] for i in training_pose_labels)
        y_train = pdbbind_training_set_pk.loc[training_pose_pdbs].values.ravel()

        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)

        # test on all docks
        fold_pose_labels = []
        for pdb in fold:
            fold_pose_labels.extend(docked_pose_labels[pdb])
        fold_pose_labels = pd.Index(fold_pose_labels)
        X_test = docked_pose_features.loc[fold_pose_labels, feature_sets[model]].values
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()

        pred = pd.Series(data=rf.predict(X_test), index=fold_pose_labels)
        max_pred = []
        for pdb in fold:
            max_pred.append(np.max(pred.loc[docked_pose_labels[pdb]]))
        fold_pearsonr.append(stats.pearsonr(y_test, max_pred)[0])
        fold_mse.append(mean_squared_error(y_test, max_pred))

    cv_train_all_docks_test_all_docks_results[model] = {'pearsonr': np.mean(fold_pearsonr), 
                                                        'rmse': np.sqrt(np.mean(fold_mse)), 
                                                        'pearsonr_stdev': np.std(fold_pearsonr)}

In [36]:
cv_train_redundant_docks_test_all_docks_results = {}

for model in feature_sets:
    fold_pearsonr = []
    fold_mse = []
    for fold in folds:

        index_train = pdbbind_training_set.difference(fold)
        
        training_pose_labels = []
        for pdb in index_train:
            training_pose_labels.extend(docked_pose_labels[pdb])
        training_pose_labels = pd.Index(training_pose_labels)

        # This time we want N copies of the pose ranked highest by Smina
        training_pose_labels = training_pose_labels.map(lambda x: x[:4]+'_0')
        X_train = docked_pose_features.loc[training_pose_labels, feature_sets[model]].values

        # Training affinities are the same for each pose of a ligand
        training_pose_pdbs = pd.Index(i[:4] for i in training_pose_labels)
        y_train = pdbbind_training_set_pk.loc[training_pose_pdbs].values.ravel()

        rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42)
        rf.fit(X_train, y_train)

        # test on all docks
        fold_pose_labels = []
        for pdb in fold:
            fold_pose_labels.extend(docked_pose_labels[pdb])
        fold_pose_labels = pd.Index(fold_pose_labels)
        X_test = docked_pose_features.loc[fold_pose_labels, feature_sets[model]].values
        y_test = pdbbind_training_set_pk.loc[fold].values.ravel()

        pred = pd.Series(data=rf.predict(X_test), index=fold_pose_labels)
        max_pred = []
        for pdb in fold:
            max_pred.append(np.max(pred.loc[docked_pose_labels[pdb]]))
        fold_pearsonr.append(stats.pearsonr(y_test, max_pred)[0])
        fold_mse.append(mean_squared_error(y_test, max_pred))
    
    cv_train_redundant_docks_test_all_docks_results[model] = {'pearsonr': np.mean(fold_pearsonr), 
                                                              'rmse': np.sqrt(np.mean(fold_mse)),
                                                              'pearsonr_stdev': np.std(fold_pearsonr)}

In [14]:
multipose_pearsonr = {}

for m in feature_sets:
    multipose_pearsonr[m] = {'Smina top pose': cv_train_top_dock_test_all_docks_max_results[m]['pearsonr'],
                                'All poses': cv_train_all_docks_test_all_docks_results[m]['pearsonr'],
                                'Redundant poses': cv_train_redundant_docks_test_all_docks_results[m]['pearsonr']}

multipose_pearsonr = pd.DataFrame(multipose_pearsonr)[['LB', 'SB', 'HB']]
multipose_pearsonr.columns = ['LB model', 'SB model','HB model']
multipose_pearsonr.to_csv('../results/train_multiple_poses_cv_pearsonr.csv')
multipose_pearsonr

,LB model,SB model,HB model
Smina top pose,0.718999,0.687132,0.744309
All poses,0.712263,0.699169,0.723782
Redundant poses,0.712263,0.687649,0.744613


In [41]:
multipose_pearsonr_stdev = {}

for m in feature_sets:
    multipose_pearsonr_stdev[m] = {'Smina top pose': cv_train_top_dock_test_all_docks_max_results[m]['pearsonr_stdev'],
                                'All poses': cv_train_all_docks_test_all_docks_results[m]['pearsonr_stdev'],
                                'Redundant poses': cv_train_redundant_docks_test_all_docks_results[m]['pearsonr_stdev']}

multipose_pearsonr_stdev = pd.DataFrame(multipose_pearsonr_stdev)[['LB', 'SB', 'HB']]
multipose_pearsonr_stdev.columns = ['LB model', 'SB model','HB model']
multipose_pearsonr_stdev

,LB model,SB model,HB model
Smina top pose,0.010191,0.013228,0.009312
All poses,0.006457,0.014924,0.007673
Redundant poses,0.006457,0.012987,0.009683


In [70]:
with open('pdbbind_training_set_crystal_rfv2.json') as f:
    rfv2_features_crystal = pd.DataFrame(json.load(f)).T
with open('pdbbind_training_set_docked_rfv2.json') as f:
    rfv2_features_docked = pd.DataFrame(json.load(f)).T

### RF-Score v2

In [75]:
%%time

lb_features = crystal_pose_features.loc[pdbbind_training_set, feature_sets['LB']]
rfv2_rdk_features_crystal = pd.concat([rfv2_features_crystal, lb_features], axis='columns')
rfv2_rdk_features_docked = pd.concat([rfv2_features_docked, lb_features], axis='columns')

rfv2_cv_crystal_results = {}
rfv2_cv_docked_results = {}
rfv2_rdk_cv_crystal_results = {}
rfv2_rdk_cv_docked_results = {}

print('RFv2 crytsal')
fold_pearsonr = []
fold_mse = []
for fold in folds:
    index_train = pdbbind_training_set.difference(fold)
    X_train = rfv2_features_crystal.loc[index_train].values
    X_test = rfv2_features_crystal.loc[fold].values
    y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()
    y_test = pdbbind_training_set_pk.loc[fold].values.ravel()
    rf = RandomForestRegressor(n_estimators=500, max_features=14, n_jobs=6, random_state=42) #mtry=14 for rfv2
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    fold_pearsonr.append(stats.pearsonr(y_test, pred)[0])
    fold_mse.append(mean_squared_error(y_test, pred))
rfv2_cv_crystal_results = {'pearsonr': np.mean(fold_pearsonr), 'rmse': np.sqrt(np.mean(fold_mse)), 'pearsonr_stdev': np.std(fold_pearsonr)}

print('RFv2 docked')
fold_pearsonr = []
fold_mse = []
for fold in folds:
    index_train = pdbbind_training_set.difference(fold)
    X_train = rfv2_features_docked.loc[index_train].values
    X_test = rfv2_features_docked.loc[fold].values
    y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()
    y_test = pdbbind_training_set_pk.loc[fold].values.ravel()
    rf = RandomForestRegressor(n_estimators=500, max_features=14, n_jobs=6, random_state=42) #mtry=14 for rfv2
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    fold_pearsonr.append(stats.pearsonr(y_test, pred)[0])
    fold_mse.append(mean_squared_error(y_test, pred))
rfv2_cv_docked_results = {'pearsonr': np.mean(fold_pearsonr), 'rmse': np.sqrt(np.mean(fold_mse)), 'pearsonr_stdev': np.std(fold_pearsonr)}

print('RFv2 rdk crytsal')
fold_pearsonr = []
fold_mse = []
for fold in folds:
    index_train = pdbbind_training_set.difference(fold)
    X_train = rfv2_rdk_features_crystal.loc[index_train].values
    X_test = rfv2_rdk_features_crystal.loc[fold].values
    y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()
    y_test = pdbbind_training_set_pk.loc[fold].values.ravel()
    rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42) #mtry=14 for rfv2
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    fold_pearsonr.append(stats.pearsonr(y_test, pred)[0])
    fold_mse.append(mean_squared_error(y_test, pred))
rfv2_rdk_cv_crystal_results = {'pearsonr': np.mean(fold_pearsonr), 'rmse': np.sqrt(np.mean(fold_mse)), 'pearsonr_stdev': np.std(fold_pearsonr)}

print('RFv2 rdk docked')
fold_pearsonr = []
fold_mse = []
for fold in folds:
    index_train = pdbbind_training_set.difference(fold)
    X_train = rfv2_rdk_features_docked.loc[index_train].values
    X_test = rfv2_rdk_features_docked.loc[fold].values
    y_train = pdbbind_training_set_pk.loc[index_train].values.ravel()
    y_test = pdbbind_training_set_pk.loc[fold].values.ravel()
    rf = RandomForestRegressor(n_estimators=500, max_features=0.33, n_jobs=6, random_state=42) #mtry=14 for rfv2
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    fold_pearsonr.append(stats.pearsonr(y_test, pred)[0])
    fold_mse.append(mean_squared_error(y_test, pred))
rfv2_rdk_cv_docked_results = {'pearsonr': np.mean(fold_pearsonr), 'rmse': np.sqrt(np.mean(fold_mse)), 'pearsonr_stdev': np.std(fold_pearsonr)}

RFv2 crytsal
RFv2 docked
RFv2 rdk crytsal
RFv2 rdk docked
CPU times: user 8min 56s, sys: 1.66 s, total: 8min 57s
Wall time: 1min 35s


In [78]:
rfv2_cv_crystal_results

{'pearsonr': 0.7464975431553001,
 'rmse': 1.306679466488912,
 'pearsonr_stdev': 0.012753270474693329}

In [79]:
rfv2_cv_docked_results

{'pearsonr': 0.7010702124420016,
 'rmse': 1.397436364008507,
 'pearsonr_stdev': 0.01038715021004822}

In [80]:
rfv2_rdk_cv_crystal_results

{'pearsonr': 0.7690245884295768,
 'rmse': 1.2543885218983784,
 'pearsonr_stdev': 0.011667248602019877}

In [82]:
rfv2_rdk_cv_docked_results

{'pearsonr': 0.7495259570487816,
 'rmse': 1.299493903757424,
 'pearsonr_stdev': 0.010746989602424534}